<a href="https://colab.research.google.com/github/kareemullah123456789/BDF-big_data_foundation_scenario-/blob/main/pyspark_service_scenarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark for Service-Based Companies (Cognizant, TCS, Infosys)

This notebook simulates real-world projects you will encounter as a Data Engineer or Data Scientist in major service-based IT companies.

## Learning Objectives
1.  **Understand the "Service" Context:** Clients (Banks, Retailers, Telecoms) hire service companies to solve specific data problems (Migration, Data Quality, AI Readiness).
2.  **Master PySpark for ETL:** 80% of AI projects start with cleaning and transforming massive datasets.
3.  **Solve Real Scenarios:** We will walk through 4 common project types:
    *   **Migration:** Moving legacy CSVs to a modern Data Lake (Parquet).
    *   **Data Quality (DQ):** Handling duplicates and bad data in Insurance claims.
    *   **Feature Engineering:** Aggregating transactional telecom data for Churn Prediction.
    *   **Log Analytics:** Parsing server logs for Fraud Detection.

---

## 1. Environment Setup (Google Colab Friendly)
Run the following cells to install Apache Spark and set up the environment.

In [ ]:
# Install necessary libraries (Only needed for Google Colab/local env without Spark pre-installed)
try:
    import pyspark
    print("PySpark is already installed.")
except ImportError:
    print("Installing PySpark...")
    !pip install pyspark findspark

# Initialize Spark Session (Entry point for all PySpark Applications)
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import os

# Create Spark Session
spark = SparkSession.builder \
    .appName("ServiceBasedCompanyScenarios") \
    .master("local[*]") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")
print("Spark Session Created Successfully!")

## Scenario 1: Legacy Data Migration (Retail Client)

**Problem Statement:**
A large retail client (e.g., Walmart) is migrating 10 years of sales data from an old Mainframe system to a modern Hadoop/Spark Data Lake. The data is received as messy CSV files.
*   **Challenge:** The data has `null` values for `store_id` (data corruption) and dates are in inconsistent formats.
*   **Goal:**
    1.  Read the raw CSV data.
    2.  Filter out corrupted records (where `store_id` is null).
    3.  Standardize the date format.
    4.  Write the clean data to **Parquet** format (columnar storage optimized for Big Data).

**Why this matters:**
This is the "Hello World" of Data Engineering projects. 90% of your first project will involve moving data from Place A to Place B and cleaning it.

**Approach:**
1.  Define a strict Schema using `StructType` to enforce data types.
2.  Use `spark.read` with options to handle headers and potential malformed rows.
3.  Use `na.drop` or `filter` to remove bad data.
4.  Use `withColumn` and `to_date` to fix date strings.
5.  Write using `mode("overwrite").parquet()`.

In [ ]:
# --- Scenario 1: Retail Data Migration & Cleaning ---

# 1. Generate Dummy Data (Simulating a CSV file from a legacy system)
retail_data_raw = [
    (101, "SKU_A", "2023-01-15", 50.0, "Completed"),
    (102, "SKU_B", "15-01-2023", 30.0, "Returned"),  # Date format issue
    (None, "SKU_C", "2023-01-16", 20.0, "Failed"),   # Corrupted ID
    (103, "SKU_D", "2023/01/17", 100.0, None),       # Null Status
    (104, "SKU_E", "2023-01-18", None, "Processing") # Null Amount
]

# We will treat this list as our "Raw" DataFrame for simulation
raw_cols = ["store_id", "product_sku", "transaction_date", "amount", "status"]
df_retail = spark.createDataFrame(retail_data_raw, schema=raw_cols)

print("--- Raw Retail Data (From Legacy System) ---")
df_retail.show()

# 2. Step: Data Cleaning
# a. Handle missing Store IDs (Critical for business logic)
df_clean = df_retail.dropna(subset=["store_id"])

# b. Handle missing Amounts (Assume 0 if missing, though typically we might flag or drop)
df_clean = df_clean.fillna(0.0, subset=["amount"])

# c. Standardize Date Format
# The raw data has mixed formats: 'yyyy-MM-dd', 'dd-MM-yyyy', 'yyyy/MM/dd'.
# In a real project, you might use complex Regex or UDFs. Here we demonstrate a simple coalescing approach.
# We will create a new column 'clean_date' using `coalesce` to try multiple formats.
df_clean = df_clean.withColumn("clean_date",
    coalesce(
        to_date(col("transaction_date"), "yyyy-MM-dd"),
        to_date(col("transaction_date"), "dd-MM-yyyy"),
        to_date(col("transaction_date"), "yyyy/MM/dd")
    )
)

# d. Filter out rows where date parsing failed (Optional, depending on business rule)
df_final_retail = df_clean.filter(col("clean_date").isNotNull())

print("--- Cleaned Retail Data (Ready for Data Lake) ---")
df_final_retail.show()

# 3. Write to Parquet (Simulated Path)
# In a real environment like HDFS or S3: output_path = "s3a://data-lake/processed/retail_sales/"
output_path = "retail_sales_processed"
df_final_retail.write.mode("overwrite").parquet(output_path)
print(f"Data successfully written to {output_path} in Parquet format.")

## Scenario 2: Data Quality Framework (Insurance Client)

**Problem Statement:**
An Insurance client (e.g., AIG, MetLife) receives thousands of claim files daily. However due to system errors, duplicate records are appearing for the same `claim_id`.
*   **Challenge:** We need the *latest* version of each claim based on timestamp.
*   **Goal:**
    1.  Identify duplicates.
    2.  Keep only the record with the most recent `update_time`.
    3.  Discard older, redundant records.

**Why this matters:**
Data Quality (DQ) is critical. If you train an ML model on duplicate data, it becomes biased. If you pay a claim twice, you lose money.

**Approach:**
1.  Use `Window` functions to partition data by `claim_id`.
2.  Order by `update_time` descending.
3.  Assign a `row_number()`.
4.  Filter to keep only rows where `row_number == 1`.
This is often called **Deduplication** or **CDC (Change Data Capture)** logic.

In [ ]:
# --- Scenario 2: Data Quality (Deduplication Logic) ---

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, desc

# 1. Generate Dummy Insurance Claims Data
# Notice 'CLAIM_101' appears twice. We want the one with the latest update_time.
claims_data = [
    ("CLAIM_101", "Auto_Accident", 5000.0, "2023-01-01 10:00:00"),
    ("CLAIM_102", "Fire_Damage", 12000.0, "2023-01-01 10:30:00"),
    ("CLAIM_101", "Auto_Accident_v2", 5000.0, "2023-01-01 11:00:00"), # Newer update
    ("CLAIM_103", "Theft", 2500.0, "2023-01-02 09:00:00"),
    ("CLAIM_103", "Theft_Correction", 2500.0, "2023-01-02 08:00:00")  # Older update (should be discarded)
]

cols = ["claim_id", "description", "claim_amount", "update_time"]
df_claims = spark.createDataFrame(claims_data, schema=cols)

print("--- Raw Claims Data (Includes Duplicates/Versions) ---")
df_claims.show(truncate=False)

# 2. Define Window Specification
# Partition by claim_id: "Group rows by claim ID"
# Order by update_time DESC: "Put the latest one at the top"
windowSpec = Window.partitionBy("claim_id").orderBy(desc("update_time"))

# 3. Apply Row Number
df_ranked = df_claims.withColumn("rn", row_number().over(windowSpec))

# 4. Filter to Keep Only Top Record (Latest Update)
df_deduped = df_ranked.filter(col("rn") == 1).drop("rn")

print("--- Deduplicated Data (Production Ready) ---")
df_deduped.show(truncate=False)

# This logic is extremely common in Incremental Loads.

## Scenario 3: Feature Engineering for Churn Prediction (Telecom Client)

**Problem Statement:**
A Telecom giant (e.g., Verizon, AT&T) wants to predict which customers will leave (churn).
*   **The Data:** You are given raw Call Detail Records (CDRs). Each row is a single call.
*   **Challenge:** ML models cannot learn from raw logs. They need a "Customer Profile" (one row per customer).
*   **Goal:** Aggregate the raw logs to create features like:
    1.  `total_calls`: Count of calls made.
    2.  `total_duration`: Sum of call minutes.
    3.  `avg_call_duration`: Average minutes per call.
    4.  `is_high_usage`: Flag if total duration > 100 mins.

**Why this matters:**
This bridges the gap between Data Engineering and Data Science. You are preparing the *Input Matrix (X)* for the model.

**Approach:**
1.  `groupBy("customer_id")`
2.  `agg(count(...), sum(...))`
3.  Use `when().otherwise()` to create categorical flags.

In [ ]:
# --- Scenario 3: Feature Engineering (Churn Prediction) ---

from pyspark.sql.functions import sum, avg, count, when

# 1. Generate Dummy Call Detail Records (CDRs)
# Each row represents one phone call by a customer.
cdr_data = [
    ("CUST_A", "2023-01-01", 10),  # 10 minutes
    ("CUST_A", "2023-01-02", 50),
    ("CUST_B", "2023-01-01", 5),
    ("CUST_A", "2023-01-03", 100), # Long call
    ("CUST_B", "2023-01-02", 5),
    ("CUST_C", "2023-01-01", 0),   # Failed call
    ("CUST_C", "2023-01-02", 1)
]

df_cdr = spark.createDataFrame(cdr_data, ["customer_id", "call_date", "duration_mins"])

print("--- Raw Telecom Logs (Per Call) ---")
df_cdr.show()

# 2. Key Aggregations (Feature Engineering)
# We need to transform this into ONE ROW PER CUSTOMER for the ML model.
df_features = df_cdr.groupBy("customer_id").agg(
    count("*").alias("total_calls"),
    sum("duration_mins").alias("total_duration_mins"),
    avg("duration_mins").alias("avg_call_duration"),
    # Advanced: Count how many long calls (> 30 mins) they made
    sum(when(col("duration_mins") > 30, 1).otherwise(0)).alias("long_calls_count")
)

# 3. Derived Features (Business Logic)
# If total_duration > 100, flag as "High Value Customer"
df_final_features = df_features.withColumn(
    "is_high_value",
    when(col("total_duration_mins") > 100, 1).otherwise(0)
)

print("--- Final Customer Features (Ready for ML Model) ---")
df_final_features.show()
# This DataFrame would be saved and then fed into a LogisticRegression or RandomForest model.

## Scenario 4: Log Analysis for Fraud Detection (Banking Client)

**Problem Statement:**
A Bank (e.g., Citibank, Chase) needs to detect if a single IP address is hammering their login page (Brute Force Attack).
*   **The Data:** Raw server logs (Apache/Nginx style). It's just text strings.
*   **Challenge:** Need to extract IP, Timestamp, and Endpoint from text.
*   **Goal:**
    1.  Parse the unstructured log.
    2.  Count login attempts per IP per minute.
    3.  Flag any IP with > 5 attempts in 1 minute.

**Why this matters:**
Cybersecurity Analytics is a huge field. Spark is perfect for parsing petabytes of access logs to find anomalies.

**Approach:**
1.  Use `regexp_extract` to pull fields from the raw string.
2.  Convert timestamp string to actual TimestampType.
3.  Use Window functions (or `groupBy` with time windows) to count events.

In [ ]:
# --- Scenario 4: Log Analysis for Fraud Detection ---

from pyspark.sql.functions import regexp_extract, count, col

# 1. Generate Log Data (Unstructured Text)
log_data = [
    ("192.168.1.10 - - [01/Jan/2023:12:00:01] \"GET /login HTTP/1.1\" 200",),
    ("192.168.1.10 - - [01/Jan/2023:12:00:02] \"POST /login HTTP/1.1\" 401",),
    ("192.168.1.10 - - [01/Jan/2023:12:00:03] \"POST /login HTTP/1.1\" 401",),
    ("192.168.1.10 - - [01/Jan/2023:12:00:04] \"POST /login HTTP/1.1\" 401",),
    ("192.168.1.10 - - [01/Jan/2023:12:00:05] \"POST /login HTTP/1.1\" 401",),
    ("192.168.1.10 - - [01/Jan/2023:12:00:06] \"POST /login HTTP/1.1\" 401",), # 6th attempt!
    ("10.0.0.5 - - [01/Jan/2023:12:05:01] \"GET /home HTTP/1.1\" 200",)      # Normal user
]

df_logs = spark.createDataFrame(log_data, ["raw_log"])
print("--- Raw Logs (Unstructured Strings) ---")
df_logs.show(truncate=False)

# 2. Parse Logs Steps
# Regex Pattern: match IP at start, then ignore up to timestamp in brackets
# IP Pattern: ^(\S+)
# Timestamp Pattern: \[(\S+)\]
# Endpoint Pattern: \"(\S+)\s(\S+)\s
ip_pattern = r'^(\S+)'
ts_pattern = r'\[(\S+)\]'
method_pattern = r'\"(\S+)\s+(\S+)\s+'

df_parsed = df_logs.withColumn("ip_address", regexp_extract("raw_log", ip_pattern, 1)) \
    .withColumn("timestamp_str", regexp_extract("raw_log", ts_pattern, 1)) \
    .withColumn("method", regexp_extract("raw_log", method_pattern, 1)) \
    .withColumn("endpoint", regexp_extract("raw_log", method_pattern, 2))

# 3. Filter for Failed Login Attempts (or just heavy traffic on /login)
df_login_attempts = df_parsed.filter((col("endpoint") == "/login"))

# 4. Detect Brute Force (Count per IP)
# In real streaming, we use windowing over time. Here, we group by IP.
df_fraud_alert = df_login_attempts.groupBy("ip_address").count() \
    .withColumnRenamed("count", "attempt_count") \
    .filter(col("attempt_count") > 5) \
    .withColumn("alert", lit("POSSIBLE BRUTE FORCE ATTACK"))

print("--- Fraud Detection Alert (High Login Failures) ---")
df_fraud_alert.show()

# If the list is empty, no fraud. In our dummy data, 192.168.1.10 has 6 hits.